In [1]:
import os.path, sys

import numpy as np
import pandas as pd
from scipy import linalg
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder

import Bio.PDB, warnings
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from scipy.spatial import distance_matrix
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)

from joblib import Parallel, delayed
import timeit

import matplotlib.pyplot as plt

# # --- Import our Code ---# #
#import emachine as EM
from direct_info import direct_info

# import data processing and general DCA_ER tools
from pathlib import Path
np.random.seed(1)

from Bio import SeqIO
from Bio.PDB import *
from scipy.spatial import distance_matrix
from Bio import pairwise2
from Bio.SubsMat.MatrixInfo import blosum62
pdb_parser = Bio.PDB.PDBParser()
from Bio.pairwise2 import format_alignment
from prody import *

print('Done with initial import')

from data_processing import pdb2msa, data_processing_pdb2msa

import ecc_tools as tools

Done with initial import


In [2]:
create_new = False
printing = True
removing_cols = True
remove_diagonals = False

data_path = Path('/data/cresswellclayec/DCA_ER/Pfam-A.full')
data_path = Path('/data/cresswellclayec/Pfam-A.full')

# Define data directories
DCA_ER_dir = '/data/cresswellclayec/DCA_ER' # Set DCA_ER directory
biowulf_dir = '%s/biowulf_full' % DCA_ER_dir


out_dir = '%s/protein_data/di/' % biowulf_dir
out_metric_dir = '%s/protein_data/metrics/' % biowulf_dir

processed_data_dir = "%s/protein_data/data_processing_output" % biowulf_dir
pdb_dir = '%s/protein_data/pdb_data/' % biowulf_dir


pdb_path = "/pdb/pdb/"
dir_path = "/data/cresswellclayec/DCA_ER/biowulf_full/protein_data/di/"

ER_result = list(Path(dir_path).rglob("*ER_di.npy"))
ER_di_pdb_list = [str(path)[-22:-18] for path in ER_result]
ER_di_pfam_list = [str(path)[-17:-10] for path in ER_result]
ER_pdb_path_str = ['%s%s/pdb%s.ent.gz' % (pdb_path, pdb[1:3], pdb) for pdb in ER_di_pdb_list]
print('ER di: ', len(ER_di_pdb_list))
print(ER_di_pdb_list[:5])
print(ER_di_pfam_list[:5])

PLM_result = list(Path(dir_path).rglob("*PLM_di.npy"))
PLM_di_pdb_list = [str(path)[-23:-19] for path in PLM_result]
PLM_di_pfam_list = [str(path)[-18:-13] for path in PLM_result]
PLM_pdb_path_str = ['%s%s/pdb%s.ent.gz' % (pdb_path, pdb[1:3], pdb) for pdb in PLM_di_pdb_list]
print('PLM di: ', len(PLM_di_pdb_list))
print(PLM_di_pdb_list[:5])
print(PLM_di_pfam_list[:5])

PMF_result = list(Path(dir_path).rglob("*PMF_di.npy"))
PMF_di_pdb_list = [str(path)[-23:-19] for path in PMF_result]
PMF_di_pfam_list = [str(path)[-18:-13] for path in PMF_result]
PMF_pdb_path_str = ['%s%s/pdb%s.ent.gz' % (pdb_path, pdb[1:3], pdb) for pdb in PMF_di_pdb_list]
print('PMF di: ', len(PMF_di_pdb_list))
print(PMF_di_pdb_list[:5])
print(PMF_di_pfam_list[:5], '\n\n')


comparison_pdb_str_set = set.intersection (set(ER_di_pdb_list), set(PLM_di_pdb_list), set(PMF_di_pdb_list))
print(len(comparison_pdb_str_set))
# print(comparison_pdb_str_set)
pdb_comparison_paths = ['%s%s/pdb%s.ent.gz' % (pdb_path,  pdb[1:3], pdb) for pdb in comparison_pdb_str_set]




ER di:  10641
['5a1g', '1bpj', '4bgz', '4am4', '2a14']
['PF02773', 'PF00303', 'PF00509', 'PF00210', 'PF01234']
PLM di:  9946
['1c3t', '1c3m', '3ba9', '6a65', '7aw6']
['PF002', 'PF014', 'PF016', 'PF003', 'PF076']
PMF di:  10632
['2a8u', '4ars', '7c4o', '4a3s', '6c7x']
['PF034', 'PF003', 'PF075', 'PF003', 'PF001'] 


9939


In [3]:
for pdb_path in pdb_comparison_paths[:10]:
    # pdb_path = "/pdb/pdb/zd/pdb1zdr.ent.gz"
#     pdb_path = "/pdb/pdb/a1/pdb5a1g.ent.gz"
    pdb_id = pdb_path[15:19]
    print(pdb_id)

    print(pdb_out_path)
    print(pdb_dir)

    prody_df = pd.read_csv('%s/%s_pdb_df.csv' % (pdb_dir, pdb_id))


    # --------------------- Data Processing (should be saving correct row!!!!) --- #
    print('\nPDB DF with associated Protein Families\n', prody_df.loc[:,  [column for column in prody_df.columns if column not in ['locations', 'PDB Sequence']]].head())

    pdb2msa_row = prody_df.iloc[0]
#     for ir, pdb2msa_row in enumerate(prody_df.iterrows()):
#         print('\n\nGetting msa with following pdb2msa entry:\n', pdb2msa_row[1])
#         try:
#             dp_result =  data_processing_pdb2msa(data_path, prody_df.iloc[pdb2msa_row[0]], gap_seqs=0.2, gap_cols=0.2, prob_low=0.004,
#                                conserved_cols=0.8, printing=False, out_dir=processed_data_dir, pdb_dir=pdb_dir, letter_format=False,
#                                remove_cols=True, create_new=True, n_cpu=min(2, n_cpus))
#             if dp_result is not None:
#                 [s0, removed_cols, s_index, tpdb, pdb_s_index] = dp_result
#                 break
#             else:
#                 continue
#         except Exception as e:
#             print('row %d got exception: ' % ir , e)
#             print('moving on.. ')
#             continue
    pfam_id = pdb2msa_row['Pfam']
    pdb_id = pdb2msa_row['PDB ID']

    if removing_cols:
        s0 = np.load("%s/%s_%s_preproc_msa.npy" % (processed_data_dir, pfam_id, pdb_id))
        s_index = np.load("%s/%s_%s_preproc_sindex.npy" % (processed_data_dir, pfam_id, pdb_id))
        pdb_s_index = np.load("%s/%s_%s_preproc_pdb_sindex.npy" % (processed_data_dir, pfam_id, pdb_id))
        removed_cols = np.load("%s/%s_%s_removed_cols.npy" % (processed_data_dir, pfam_id, pdb_id))
        ref_seq = np.load("%s/%s_%s_preproc_refseq.npy" % (processed_data_dir, pfam_id, pdb_id))
    else:
        s0 = np.load("%s/%s_%s_allCols_msa.npy" % (processed_data_dir, pfam_id, pdb_id))
        s_index = np.load("%s/%s_%s_allCols_sindex.npy" % (processed_data_dir, pfam_id, pdb_id))
        pdb_s_index = np.load("%s/%s_%s_allCols_pdb_sindex.npy" % (processed_data_dir, pfam_id, pdb_id))
        removed_cols = np.load("%s/%s_%s_removed_cols.npy" % (processed_data_dir, pfam_id, pdb_id))
        ref_seq = np.load("%s/%s_%s_allCols_refseq.npy" % (processed_data_dir, pfam_id, pdb_id))



    print('Done found correct prody_df row for contact predictions ... NOTE\n this step should be saved during data_processing!!!...')
    # ---------------------------------------------------------------------------- #

    print('\n\n...Done data_processing\n\n')


6c1e


NameError: name 'pdb_out_path' is not defined

In [ ]:
ER_di = np.load("%s/%s_%s_ER_di.npy" % (out_dir, pdb_id, pfam_id))
MF_di = np.load("%s/%s_%s_MF_di.npy" % (out_dir, pdb_id, pfam_id))
PMF_di_data = np.load("%s/%s_%s_PMF_di.npy" % (out_dir, pdb_id, pfam_id),allow_pickle=True)
PLM_di_data = np.load("%s/%s_%s_PLM_di.npy" % (out_dir, pdb_id, pfam_id),allow_pickle=True)
#print("Direct Information from Expectation reflection:\n",di)
print('s_index (%d): ' %len(s_index), s_index)
print('ER DI shape (before removing cols): ' , ER_di.shape)
print('MF DI shape (before removing cols): ' , MF_di.shape)
print('PMF DI shape (before removing cols): ' , PMF_di_data.shape)
print('PLM_DI shape (before removing cols): ' , PLM_di_data.shape)

ct, ct_full = tools.contact_map_pdb2msa_new(pdb2msa_row[1], pdb_out_path, removed_cols, pdb_s_index, pdb_out_dir=pdb_dir, printing=True)


er_di = ER_di
#if not removing_cols:
#    er_di = np.delete(di, removed_cols_range,0)
#    er_di = np.delete(er_di, removed_cols_range,1)
#else:
#    er_di = di

print('contact dimentions: ', ct.shape)
print('Final ER DI shape (cols removed): ', er_di.shape)
if remove_diagonals:
    ER_di = no_diag(er_di, 4, s_index)
else:
    ER_di = er_di

from ecc_tools import scores_matrix2dict
print(s_index)
#ER_di_dict = scores_matrix2dict(ER_di, s_index, removed_cols_range, removing_cols=removing_cols)
#print(ER_di_dict[:20])


from ecc_tools import roc_curve, roc_curve_new, precision_curve
# find optimal threshold of distance
ct_thres = np.linspace(4.,6.,18,endpoint=True)
n = ct_thres.shape[0]



ct_mat = ct


# ER ROC
auc_ER = np.zeros(n)
for i in range(n):
    try:
        fpr, tpr, thresholds, auc = roc_curve_new(ct_mat, ER_di, ct_thres[i])
        auc_ER[i] = auc
    except:
        auc_ER[i] = 0

# Get ER method's best contact prediction
i0_ER = np.argmax(auc_ER)
print('ER auc max:',ct_thres[i0_ER],auc_ER[i0_ER])
fpr0_ER, tpr0_ER, thresholds_ER, auc = roc_curve_new(ct_mat, ER_di, ct_thres[i0_ER])

fp_file = "%s%s_%s_ER_fp.npy" % (out_metric_dir, pdb_id, pfam_id)
tp_file = "%s%s_%s_ER_tp.npy" % (out_metric_dir, pdb_id, pfam_id)
np.save(fp_file, fpr0_ER)
np.save(tp_file, tpr0_ER)

# # MF ROC
# auc_MF = np.zeros(n)
# for i in range(n):
#     try:
#         fpr, tpr, thresholds, auc = roc_curve_new(ct_mat, MF_di, ct_thres[i])
#         auc_MF[i] = auc
#     except:
#         auc_MF[i] = 0

# # Get ER method's best contact prediction
# i0_MF = np.argmax(auc_MF)
# print('MF auc max:',ct_thres[i0_MF],auc_MF[i0_MF])
# fpr0_MF, tpr0_MF, thresholds_MF, auc = roc_curve_new(ct_mat, MF_di, ct_thres[i0_MF])

# fp_file = "%s%s_%s_MF_fp.npy" % (out_metric_dir, pdb_id, pfam_id)
# tp_file = "%s%s_%s_MF_tp.npy" % (out_metric_dir, pdb_id, pfam_id)
# np.save(fp_file, fpr0_ER)
# np.save(tp_file, tpr0_ER)


# PMF ROC
# translate PMF di tuple to contact matrix
PMF_di = np.zeros(ER_di.shape)
PMF_di_dict = {}
for score_set in PMF_di_data:
    PMF_di_dict[(score_set[0][0], score_set[0][1])] = score_set[1]
for i, index_i in enumerate(s_index):
    for j, index_j in enumerate(s_index):
        if i==j:
            PMF_di[i,j] = 1.
            continue
        try:
            PMF_di[i,j] = PMF_di_dict[(index_i, index_j)]
            PMF_di[j,i] = PMF_di_dict[(index_i, index_j)] # symetric
        except(KeyError):
            continue


auc_PMF = np.zeros(n)
for i in range(n):
    try:
        fpr, tpr, thresholds, auc = roc_curve_new(ct_mat, PMF_di, ct_thres[i])
        auc_PMF[i] = auc
    except:
        auc_PMF[i] = 0

# Get ER method's best contact prediction
i0_PMF = np.argmax(auc_PMF)
print('PMF auc max:',ct_thres[i0_PMF],auc_PMF[i0_PMF])
fpr0_PMF, tpr0_PMF, thresholds_PMF, auc = roc_curve_new(ct_mat, PMF_di, ct_thres[i0_PMF])

fp_file = "%s%s_%s_PMF_fp.npy" % (out_metric_dir, pdb_id, pfam_id)
tp_file = "%s%s_%s_PMF_tp.npy" % (out_metric_dir, pdb_id, pfam_id)
np.save(fp_file, fpr0_PMF)
np.save(tp_file, tpr0_PMF)


# PLM ROC
# translate PMF di tuple to contact matrix
PLM_di = np.zeros(ER_di.shape)
PLM_di_dict = {}
for score_set in PLM_di_data:
    PLM_di_dict[(score_set[0][0], score_set[0][1])] = score_set[1]
for i, index_i in enumerate(s_index):
    for j, index_j in enumerate(s_index):
        if i==j:
            PLM_di[i,j] = 1.
            continue
        try:
            PLM_di[i,j] = PLM_di_dict[(index_i, index_j)]
            PLM_di[j,i] = PLM_di_dict[(index_i, index_j)] # symetric
        except(KeyError):
            continue



auc_PLM = np.zeros(n)
for i in range(n):
    try:
        fpr, tpr, thresholds, auc = roc_curve_new(ct_mat, PLM_di, ct_thres[i])
        auc_PLM[i] = auc
    except:
        auc_PLM[i] = 0

# Get ER method's best contact prediction
i0_PLM = np.argmax(auc_PLM)
print('PLM auc max:',ct_thres[i0_PLM],auc_PLM[i0_PLM])
fpr0_PLM, tpr0_PLM, thresholds_PLM, auc = roc_curve_new(ct_mat, PLM_di, ct_thres[i0_PLM])

fp_file = "%s%s_%s_PLM_fp.npy" % (out_metric_dir, pdb_id, pfam_id)
tp_file = "%s%s_%s_PLM_tp.npy" % (out_metric_dir, pdb_id, pfam_id)
np.save(fp_file, fpr0_PLM)
np.save(tp_file, tpr0_PLM)


In [ ]:
tprs = [tpr0_ER, tpr0_MF, tpr0_PMF, tpr0_PLM]
fprs = [fpr0_ER, fpr0_MF, fpr0_PMF, fpr0_PLM]
methods = ['ER', 'MF', 'PMF', 'PLM']
plt.figure(figsize=(27.0,12.8))

# Initalize ROC-curve tile
plt.subplot2grid((1,4),(0,0))
plt.title('ROC Summary')
plt.plot([0,1],[0,1],'k--')

for i, tpr in enumerate(tprs):
    fpr = fprs[i]
    try:
        plt.plot(fpr, tpr,label=methods[i])
    except:
        pass
plt.legend()
plt.show()